1. 修改内容：使用cuOpt实现GPU加速。

In [1]:
%load_ext autoreload
%autoreload 2
from helpful_files.pentropy_v3 import *
import itertools
import json
import matplotlib.pyplot as plt
import copy
import random
# random.seed(42)
import numpy as np
# np.random.seed(42)
import time
from collections import defaultdict, Counter
start_time = time.time()

In [2]:
def Regions2Matrix(entropydict, regions):
    """
    Convert regions to a matrix in the form of list.
    """
    ine_constraints = []
    ent_num = len(entropydict.redict) + 3
    for expr in regions.exprs:
        row = [0] * ent_num
        for term in expr.terms:
            row[entropydict[term.to_ent_str()]] = term.coef
        row[-1] = expr.value
        non_zero_values = [i for i in row if i != 0]
        non_zero_count = len(non_zero_values)
        if non_zero_count == 0:
            print("0 row")
            print(expr)
        elif sum(row[:-1]) < 0:
            print("negative row")
            print(expr)
        else:
            ine_constraints.append(row)
    return ine_constraints

In [3]:
# 首先引入一些cutsetbound需要的函数
def preprocessing_single(x_vars,y_vars,z_vars,expand_vars):
    for index,x in enumerate(x_vars):
        y = y_vars[index]
        new_var = x + y
        new_var = Iutils.sort_elements(new_var)
        if new_var not in expand_vars:
            expand_vars.append(new_var)
        
        if len(z_vars) != 0:
            z = z_vars[index]
            if len(z) != 0:
                new_var = x + z
                new_var = Iutils.sort_elements(new_var)
                if new_var not in expand_vars:
                    expand_vars.append(new_var)
                
                new_var = x + y + z
                new_var = Iutils.sort_elements(new_var)
                if new_var not in expand_vars:
                    expand_vars.append(new_var)

def generate_single_inequalities(ix_vars,iy_vars,iz_vars,hx_vars,hy_vars,entropydict,regions):
    """
        更新不等式集regions，由变量生成不等式
    
        :param vars: I(x;y|z)中的z集合
        :param single_vars: 单变量集合，用于生成排列组合(x,y)
        :param  entropydict: 熵字典，存储所有互信息扩展后的联合熵变量的值，
        保证优化变量与不等式矩阵一一对应
        :param regions: 不等式集，调用函数前为上一轮迭代得到的有效不等式，
        调用函数后增加新变量生成的不等式

    """
    # generate I(x;y) & I(x;y|z)
    for index,x in enumerate(ix_vars):
        y = iy_vars[index]
        x = Ivar.rv(x)
        y = Ivar.rv(y)
        
        # print("XY",x,y,z)
        term_x = Comp(set(x))
        term_y = Comp(set(y))
        # I(x;y)
        if len(iz_vars) == 0:
            term = Term.I(term_x, term_y)
        
        # I(x;y|z) or I(x;y)
        else:
            z = iz_vars[index]
            if len(z) == 0:
                term = Term.I(term_x, term_y)
            else:
                z = Ivar.rv(z)
                term_z = Comp(set(z))
                term = Term.Ic(term_x, term_y, term_z)
            
        terms = [term]
        print(term)
        expr = Expr.empty()
        regions.append_expr(expr.inequality(terms=terms, edict=entropydict))

    # generate H(x;y)
    for index,x in enumerate(hx_vars):
        y = hy_vars[index]
        x = Ivar.rv(x)
        y = Ivar.rv(y)
        
        term_x = Comp(set(x))
        term_y = Comp(set(y))
        term = Term.Hc(term_x, term_y)

        terms = [term]
        print(term)
        expr = Expr.empty()
        regions.append_expr(expr.inequality(terms=terms, edict=entropydict))

def create_cutset_bound(N,K,user_perm,file_perm,Wkey):
    regions = Region.empty()
    entropydict = EntropyEqDict() # 因为一开始就是这样初始化的所以就不输入一个region了。

    ix_vars = []
    iy_vars = []
    iz_vars = []
    hx_vars = []
    hy_vars = []

    for s in range(1,min(N,K)+1):
        print(s)
        X_list = []
        coef = N // s
        i = 0
        for num in range(coef):
            X_comb = []
            for j in range(K):
                X_comb.append(((i + j) % N) + 1)
            X_comb_str = "".join(map(str,X_comb))
            X_list.append("X" + X_comb_str)
            i += s
        Z_list = ["Z" + str(i) for i in range(1,s+1)]
        cutset_list = X_list + Z_list

        print(cutset_list)
        
        for index,var in enumerate(cutset_list):
            if index != 0:
                ix_vars.append([var])
                if [var] not in vars:
                    vars.append([var])
                y_var = cutset_list[:index]
                iy_vars.append(y_var)
                if y_var not in vars:
                    vars.append(y_var)
        
        hx_vars.append(cutset_list)
        keyZ_list = [elem for elem in cutset_list if elem.startswith("Z")]
        keyX_list = [elem for elem in cutset_list if elem.startswith("X")]
        keyW_new_set = set()
        for keyZ in keyZ_list:
            userindex = int(keyZ[1])
            fileindex_set = set(['W'+ s[userindex] for s in keyX_list])
            keyW_new_set = keyW_new_set.union(fileindex_set)
        hy_vars.append(list(keyW_new_set))
        vars.append(list(keyW_new_set))

        print("cutset_list",cutset_list)
        print("ix_vars",ix_vars)
        print("iy_vars",iy_vars)
        print("hx_vars",hx_vars)
        print("hy_vars",hy_vars)

    print(len(vars))
    print(vars)

    expand_vars = vars[:]
    preprocessing_single(ix_vars+hx_vars,iy_vars+hy_vars,iz_vars,expand_vars)
    print("len of vars",len(vars))
    print("len of expand vars",len(expand_vars))
    # print("expand_vars",expand_vars)
    Xrvs_cons = []
    Wrvs_cons = []
    Iutils.symmetrize(user_perm,file_perm,expand_vars,entropydict,Xrvs_cons,Wrvs_cons)
    Iutils.problem_constraints_process(Wkey,entropydict)
    entropydict.regenerate_keys()
    # print(len(entropydict.redict))
    # print(entropydict.redict)
    # print(entropydict)
    print("Xrvs_cons",Xrvs_cons)
    print("Wrvs_cons",Wrvs_cons)
    generate_single_inequalities(ix_vars,iy_vars,iz_vars,hx_vars,hy_vars,entropydict,regions)

    regions.sort_exprs()
    cutset_regions = regions.copy()

    for expr in cutset_regions.exprs:
        print(expr)
    print("num of exprs",len(cutset_regions.exprs))
    regions.reduce_redundant_expr()

    return regions, entropydict

In [4]:
# define parameters
N = 2
K = 2
point_num = 12 # 采样点密度, 也就是每个单位长度的点个数
generate_size = 30 # 这个用来约束使用的z的个数, 不超过30
subset_size = 30
comb_size = 150
user_perm = list(itertools.permutations(range(1, K + 1)))
file_perm = list(itertools.permutations(range(1, N + 1)))

In [5]:
def calculate_square(plot_data):
    """
    输入要绘制的点， 就可以得到对应的面积了（也就是折线下面的面积啊）。
    """
    x_data = [item[0] for item in plot_data]
    y_data = [item[1] for item in plot_data]
    square = 0
    for i in range(1,len(x_data)):
        square += (x_data[i] - x_data[i-1]) * (y_data[i] + y_data[i-1])
    return square / 2
class Reward:
    def __init__(self, cut_plot_data,gamma = 0.99):
        self.reward_list = []
        self.last_square = calculate_square(cut_plot_data)
        self.discount_reward_list = []
        self.gamma = gamma
        self.factor = 1
    def update_reward_list(self,plot_data):
        current_square = calculate_square(plot_data)
        reward = np.exp(current_square) - np.exp(self.last_square)
        self.last_square = current_square
        self.reward_list.append(reward)
        self.discount_reward_list.append(reward * self.factor)
        self.factor *= self.gamma
        return self.discount_reward_list[-1]


In [6]:
# generate all random variables, stores in single_vars

single_vars = []
vars = [] # [["W1","W2"]]
Wrvs = [] # ["W1","W2"]
necessary_vars = [] # single + vars
W_combinations = [] # [["W1"],["W2"],["W1","W2"]]
for i in range(1, N+1):
    single_vars.append("W" + str(i))
    Wrvs.append("W" + str(i))
for i in range(1, K+1):
    single_vars.append("Z" + str(i))
X_combinations = ["X" + item for item in Iutils.generate_combinations(N, K)]
if N == 2 and K == 2:
    X_combinations = ["X12"]
elif N == 2 and K == 3:
    X_combinations = ["X112"]
elif N == 3 and K == 2:
    X_combinations = ["X12","X13"]
elif N == 3 and K == 3:
    X_combinations = ["X112","X113","X123"]
elif N == 2 and K == 4:
    X_combinations = ["X1112","X1122"]
elif N == 4 and K == 2:
    X_combinations = ["X12","X13","X14"]
elif N == 4 and K == 3:
    X_combinations = ["X112","X113","X114","X123"]
elif N == 4 and K == 4:
    X_combinations = ["X1112","X1123","X1234"]
elif N == 6 and K == 4:
    X_combinations = ["X1112","X1123","X1234"]
Xrvs_cons = Iutils.symmetry_vars(user_perm,file_perm,X_combinations) # 约束之后的结果. 还不知道是做什么用的, 现在.

for item in X_combinations:
    single_vars.append(item)
for var in single_vars:
    # vars.append([var])
    necessary_vars.append([var])
vars.append(Wrvs)
necessary_vars.append(Wrvs)
for r in range(N+1):
    # 生成指定长度的所有组合
    combos = itertools.combinations(Wrvs, r+1)
    for combo in combos:
        W_combinations.append(list(combo))
Wrvs_cons = Iutils.symmetry_vars(user_perm,file_perm,W_combinations) # 不同熵, 只保留一个.
print(Wrvs_cons)
print(single_vars)
Wkey = ','.join(sorted(Wrvs, key=Iutils.sort_key))
print(Wkey)
print(Xrvs_cons)

['W1', 'W1,W2']
['W1', 'W2', 'Z1', 'Z2', 'X12']
W1,W2
['X12']


## 新添加的用于cutsetbound的部分啊。

In [7]:
regions,entropydict = create_cutset_bound(N,K,user_perm,file_perm,Wkey)

1
['X12', 'X21', 'Z1']
cutset_list ['X12', 'X21', 'Z1']
ix_vars [['X21'], ['Z1']]
iy_vars [['X12'], ['X12', 'X21']]
hx_vars [['X12', 'X21', 'Z1']]
hy_vars [['W1', 'W2']]
2
['X12', 'Z1', 'Z2']
cutset_list ['X12', 'Z1', 'Z2']
ix_vars [['X21'], ['Z1'], ['Z1'], ['Z2']]
iy_vars [['X12'], ['X12', 'X21'], ['X12'], ['X12', 'Z1']]
hx_vars [['X12', 'X21', 'Z1'], ['X12', 'Z1', 'Z2']]
hy_vars [['W1', 'W2'], ['W1', 'W2']]
9
[['W1', 'W2'], ['X21'], ['X12'], ['Z1'], ['X12', 'X21'], ['W1', 'W2'], ['Z2'], ['X12', 'Z1'], ['W1', 'W2']]
len of vars 9
len of expand vars 13
symmetrize
symmetrize time:0.001001596450805664
number of varibles 9
Xrvs_cons ['X12']
Wrvs_cons ['W1,W2']
I(X21;X12)
I(Z1;X12,X21)
I(Z1;X12)
I(Z2;X12,Z1)
H(X12,X21,Z1|W1,W2)
H(X12,Z1,Z2|W1,W2)
2H({X12}) - H({X12,X21}) >= 0
H({Z1}) + H({X12,X21}) - H({W1,W2}) >= 0
H({X12}) + H({Z1}) - H({X12,Z1}) >= 0
H({Z1}) + H({X12,Z1}) - H({W1,W2}) >= 0
num of exprs 4


In [8]:
# vars = [] # 随机生成的变量集合,初始的z
# entropydict = EntropyEqDict()
# index = 0
# episode = 0

# necessary_vars = [] # 用来存储需要的变量集合.
# for var in single_vars:
#     vars.append([var])
#     necessary_vars.append([var])
# # necessary_vars = Iutils.symmetry_vars(N,K,vars)
# necessary_vars.append(Wrvs)

# print(necessary_vars)
# vars.append(Wrvs)

# sets = Iutils.generate_random_subsets(single_vars,subset_size,2,episode+4)
# # print(sets)
# vars += sets
# print(vars)

In [9]:
# print(Wrvs)

In [10]:
# expand_vars = vars[:] # 用来做变量扩充的, 防止后面用到的变量不存在于字典中.

# combs,combinations = Iutils.generate_combs(single_vars,comb_size)
# Iutils.preprocessing_combs(vars,single_vars,expand_vars,combs)

# entropydict = EntropyEqDict()
# entropydict_all = EntropyEqDict()

# Iutils.symmetrize_by_dict(user_perm,file_perm,expand_vars,entropydict,entropydict_all)


In [11]:

# Iutils.problem_constraints_process(Wkey,entropydict)
# entropydict.regenerate_keys()

# print(len(entropydict.redict))

In [12]:
# # generate inequalities
# regions = Region.empty()
# Iutils.generate_inequalities_combs(vars,entropydict,regions,combinations)

In [13]:
print(len(regions.exprs))
for expr in regions.exprs:
    expr.sort_terms()
    # print(expr)+

4


In [14]:
# construct the constranits matrix
ine_constraints = Regions2Matrix(entropydict, regions)
ent_num = len(entropydict.redict) + 3

In [15]:
def AddProblemConstraints2Matrix(Xrvs_cons, Wrvs_cons, entropydict, ine_constraints, ent_num):
    # R >= H(X)
    prob_cons_num = 0
    for key in Xrvs_cons:
        if entropydict.get(key) != None:
            row3 = [0] * ent_num
            row3[entropydict[key]] = -1
            row3[-2] = 1
            ine_constraints.append(row3)
            prob_cons_num += 1

    # M >= H(Z)
    row5 = [0] * ent_num
    row5[entropydict["Z1"]] = -1
    row5[-3] = 1
    ine_constraints.append(row5)
    prob_cons_num += 1

    # H(W1,..,Wn) >= n
    for key in Wrvs_cons:
        if entropydict.get(key) != None:
            rvs = key.split(",")
            row3 = [0] * ent_num
            row3[entropydict[key]] = 1
            row3[-1] = len(rvs)
            # print(row3)
            ine_constraints.append(row3)
            prob_cons_num += 1

    # M = M_value
    row5 = [0] * ent_num
    ine_constraints.append(row5)

    return ine_constraints, prob_cons_num

In [16]:
ine_constraints, prob_cons_num = AddProblemConstraints2Matrix(Xrvs_cons, Wrvs_cons, entropydict, ine_constraints, ent_num)

In [17]:
ine_constraints = np.array(ine_constraints)
print(ine_constraints.shape)
ine_constraints = ine_constraints.astype(np.float64)
expr_num = ine_constraints.shape[0] - 1 # 不等式数量
ent_num = ine_constraints.shape[1] - 1 # 减之前用于生成矩阵，减之后就是实际的熵变量数量

ori_obj_coef = np.zeros(ent_num)
ori_obj_coef[-1] = 1
print(ent_num)

(8, 8)
7


In [18]:
# original problem solver
from gurobipy import Model, GRB, LinExpr, GurobiError
import gurobipy as gp

def gurobi_solver():
    try:
        global effective_idx_gurobi
        global ori_obj_coef
        ine_list = []
        # global result_slope
        # 创建 Gurobi 模型
        model = Model("entropy_minimization")

        # 禁用输出日志（可选）
        #model.setParam("OutputFlag", 0)

        # 创建变量
        variables = []
        for i in range(ent_num):
            var = model.addVar(name=f"V{i}", lb=0)
            variables.append(var)
        obj_expr = gp.quicksum(ori_obj_coef * variables)
        model.setObjective(obj_expr, GRB.MINIMIZE)
        # model.setObjective(variables[-1], GRB.MINIMIZE)
        # var_names = ["V" + str(i) for i in range(ent_num)]
        # variables = model.addVars(var_names, lb=0, obj=[0.0] * (ent_num), name="V")
        # variables[var_names[-1]].obj = 1.0  # 设置目标函数的系数
        # model.setObjective(variables[var_names[-1]], GRB.MINIMIZE)

        # 添加不等式约束
        for ine in ine_constraints[:-1]:
            model.addConstr(LinExpr(ine[:-1], variables) >= ine[-1])

        # 添加等式约束 M = M_value
        M_cons = ine_constraints[-1]
        model.addConstr(LinExpr(M_cons[:-1], variables) == M_cons[-1])
        # 添加等式约束 M = M_value
        # model.addConstr(variables[var_names[-2]] == M_value)

        model.optimize()

        # 检查求解状态
        if model.status == GRB.OPTIMAL:
            dual_values = model.getAttr('Pi', model.getConstrs())

            indices = [i for i, val in enumerate(dual_values) if abs(val) > 1e-5 and i < len(regions.exprs)]
            # slope = dual_values[-1]
            # result_slope.append(slope)

            for index in indices:
                if index not in effective_idx_gurobi:
                    effective_idx_gurobi.append(index)
                    effective_idx_gurobi.sort()
            
            # 获取目标函数值
            optimal_value = model.objVal
            print(f"Optimal value: {optimal_value}")
        
            return optimal_value
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible. Trying to find the IIS...")
            model.computeIIS()
            model.write("model.ilp")
            print("IIS written to model.ilp. Check this file for conflicting constraints.")
            for c in model.getConstrs():
                if c.IISConstr:
                    print(f"约束 {c.constrName} 导致不可行")
                    ine_list.append(c.constrName)
            for v in model.getVars():
                if v.IISLB or v.IISUB:
                    print(f"变量 {v.varName} 的边界条件导致不可行")
            return ine_list
        else:
            print(f"Model status: {model.status}")
            return None
    except GurobiError as e:
        print(f"Gurobi Error: {e}")
        return None


In [19]:
def dual_solver():
    """
    作用: 二次规划求解器, 输入为主问题的线性规划模型, 输出为主问题的最优解和可行解的索引

    :return: 主问题的最优解和可行解的索引
    :rtype: tuple
    
    """
    try:
        global expr_num
        global dual_obj_coef
        global trans_ine_cons
        effective_idx_dual = []
        # 创建 Gurobi 模型
        model = Model("secondary LP")

        # 启用输出日志
        model.setParam("OutputFlag", 0)

        # 创建变量
        variables = []
        for i in range(expr_num):
            var = model.addVar(name=f"Y{i}", lb=0)
            variables.append(var)
        var_z = model.addVar(name="Z",lb=-GRB.INFINITY, ub=GRB.INFINITY)
        variables.append(var_z)
        # print("expr_num",expr_num)
        # print("len_var",len(variables))
        # objective_expr = gp.quicksum(coef_dual[i] * variables[i] for i in range(expr_num)) + M_value * variables[-1]
        objective_expr = gp.quicksum(dual_obj_coef * variables)
        # objective_expr = gp.quicksum(variables)
        model.setObjective(objective_expr, GRB.MAXIMIZE)

        # 添加不等式约束
        for ine in trans_ine_cons:
            model.addConstr(LinExpr(ine[:-1], variables) == ine[-1])

        # 添加等式约束 M = M_value
        # model.addConstr(variables[-1] == M_value)

        model.optimize()

        # 检查求解状态
        if model.status == GRB.OPTIMAL:
            # 获取目标函数值
            optimal_value = model.objVal
            # print(f"Optimal value: {optimal_value}")

            # 获取变量的最优解
            solution_values = {var.varName: var.x for var in model.getVars()}
            # print(f"Solution: {solution_values}")

            for var_name, var_value in solution_values.items():
                if var_value > 0:
                    index = int(var_name[1:])
                    if index < expr_num - prob_cons_num and index is not None:
                        effective_idx_dual.append(index)
            return solution_values, effective_idx_dual
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible. Trying to find the IIS...")
            model.computeIIS()
            model.write("model.ilp")
            print("IIS written to model.ilp. Check this file for conflicting constraints.")
            return None,None
        else:
            print(f"Model status: {model.status}")
            return None,None
    except GurobiError as e:
        print(f"Gurobi Error: {e}")
        return None,None

In [20]:
def find_min_effective_indices(dual_value):
    """
    用来找到全部的直线的有效不等式索引, 已经去重和排序.

    :param dual_value: 双变量的系数矩阵
    :return: 有效不等式索引的列表

    """
    non_zero_counts = np.count_nonzero(dual_value, axis=1)
    old_slope = 0
    single_cut_idx = []
    all_cut_idx = set()
    old_slope = dual_value[0][-1]
    min_row = 0
    min_cnt = non_zero_counts[0]
    for idx,row in enumerate(dual_value):
        if row[-1] != old_slope:
            for i,value in enumerate(dual_value[min_row]):
                if value > 0 and i < len(regions.exprs):
                    single_cut_idx.append(i)
            # print(f"slope:{old_slope},index{single_cut_idx}")
            all_cut_idx = all_cut_idx.union(set(single_cut_idx))
            old_slope = row[-1]
            min_cnt = non_zero_counts[idx]
            min_row = idx
            single_cut_idx = []
            continue
        if non_zero_counts[idx] < min_cnt:
            min_cnt = non_zero_counts[idx]
            min_row = idx
    for i,value in enumerate(dual_value[min_row]):
        if value > 0 and i < len(regions.exprs):
            single_cut_idx.append(i)
    all_cut_idx = all_cut_idx.union(set(single_cut_idx))
    # print(f"slope:{old_slope},index{single_cut_idx}")
    # print("all",all_cut_idx)
    return sorted(list(all_cut_idx))

In [21]:
# solve problem
plot_data = [] # 存放M值和对应的结果
effective_idx_gurobi = [] # 有效不等式的索引
all_eff_indx = set() # 所有有效不等式的索引
result_slope = [] # 斜率
ori_slope = [] # 原目标函数的斜率
M_space = np.linspace(0,N,point_num*N+1)
dual_value = [] # 存放对偶问题的解
effective_idx_cut = [] # 存放切割点的索引
# print(M_space)
for M_value in M_space:
    # 根据M_value更新约束矩阵，添加等式约束
    ine_constraints = list(ine_constraints[:-1])
    row = [0] * (ent_num + 1)
    row[-3] = 1
    row[-1] = M_value
    ine_constraints.append(row)
    ine_constraints = np.array(ine_constraints)
    
    # print("ine_constraints")
    # print(ine_constraints)
    # print("ori_obj_coef",ori_obj_coef)

    # # 更新对偶问题约束矩阵
    expr_num = ine_constraints.shape[0] - 1
    trans_ine_cons = ine_constraints.T[:-1] # 对偶问题的约束矩阵 是原约束矩阵的转置
    dual_obj_coef = ine_constraints[:,-1] # 原约束的常量 是对偶问题目标函数的系数
    trans_ine_cons = np.hstack((trans_ine_cons, ori_obj_coef.T.reshape(-1, 1))) # 原目标函数的系数，是对偶问题约束的常量
    # print("shape",trans_ine_cons.shape)
    # print("trans_ine_cons")
    # print(trans_ine_cons)
    # print("dual_obj_coef",dual_obj_coef)

    # 求解原LP问题
    result = gurobi_solver()
    if type(result) == list:
        bad = Region.empty()
        for ine in result:
            idx = int(ine[1:])
            print(f"type:{type(idx)},value:{idx}")
            terms = []
            row = ine_constraints[idx]
            for i in range(len(row) - 1):
                coef = row[i]
                if coef != 0:
                    if entropydict.get_keys_by_value(i) != None:
                        term_x = entropydict.get_keys_by_value(i)[0]
                        var_str = term_x.split(",")
                        if var_str not in vars:
                            vars.append(var_str) # 添加有效不等式中的变量
                        term_x = Comp.jes(term_x)
                        term = Term(x=[term_x.copy()],coef=int(coef),termtype=TermType.H)
                        terms.append(term) 
            expr = Expr(terms, eqtype="ge", value=row[-1])
            expr.sort_terms()
            # print("expr",expr)
            bad.append_expr(expr)
        for expr in bad.exprs:
            print(expr)
   
    # 求解对偶LP问题
    if result != 0:
        solution_values, effective_idx_dual = dual_solver()
        if solution_values is not None:
            dual_value.append(list(solution_values.values()))

        # print("effective_indices",effective_idx_dual)
        if effective_idx_dual is not None:
            all_eff_indx = all_eff_indx.union(set(effective_idx_dual))
        effective_idx_dual = sorted(list(all_eff_indx))
    
    plot_data.append((M_value, result))
if len(dual_value) > 0:
    effective_idx_cut = find_min_effective_indices(dual_value)
    # print("dual")
    # print(dual_value)
    # print(f"dual:{len(effective_idx_dual)}")
    # print(effective_idx_dual)
    # print(f"gurobi:{len(effective_idx_gurobi)}")
    # print(effective_idx_gurobi)
    # print(f"cut:{len(effective_idx_cut)}")
    # print(effective_idx_cut)

Set parameter Username
Set parameter LicenseID to value 2626985
Academic license - for non-commercial use only - expires 2026-02-24
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22635.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11260H @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 7 columns and 17 nonzeros
Model fingerprint: 0xf230f346
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 8 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+00
Optimal value: 2.0
Gurobi Optimizer versio

In [22]:
# plot

Iutils.plot_inner_bound(N,K)
# plot the calculation result
x = [item[0] for item in plot_data]
y = [item[1] for item in plot_data]
result_slope = Iutils.compute_slopes(x,y)
point_x = []
point_y = []
for i in range(1,len(result_slope)):
    if result_slope[i-1] != result_slope[i]:
        point_x.append(x[i])
        point_y.append(y[i])
plt.scatter(point_x,point_y,color='red')

for xi, yi in zip(point_x, point_y):
    label = f"({round(xi,3)}, {round(yi,3)})"
    plt.annotate(label,  
                (xi, yi),  
                textcoords="offset points",  
                xytext=(30, 5),  
                ha='center') 
plt.plot(x, y, color='red', linewidth=3, label='computed outer bound')
Iutils.plot_cutset_bound(N,K,point_num)

plt.xlim(left=0)
plt.ylim(bottom=0)
plt.xlabel("M")
plt.ylabel("R")
plt.legend()
plt.title(f"Case(N,K)=({N},{K}),episode0\nvars:{len(vars)},cons:{ine_constraints.shape}")
plt.show()
rewarder = Reward(plot_data,gamma = 0.99)

C:\Users\user\AppData\Local\Temp\ipykernel_18008\116124628.py:32: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [23]:
episode = 0
oriall_vars = [] # 存放所有已经生成过的变量
same_times = 0
a = 3
while episode <= 4:
    episode += 1
    print("episode",episode)
    print(ori_slope)
    print(result_slope)
    if ori_slope ==  result_slope:
        same_times += 1
    else:
        same_times = 0
    if same_times == a:
        break
    print("oriall vars",len(oriall_vars))
    print("same times",same_times)
    # print(entropydict_all.eqdict)
    # 更新regions、vars
    print("1.更新regions、vars")
    print("number of effective exprs:",len(effective_idx_cut))
    # print(effective_indices)
    effective_constraints = [ine_constraints[i] for i in effective_idx_cut]
    regions = Region.empty() # 更新regions
    ori_vars = vars[:]
    oriall_vars += vars[:]
    entropydict_all = entropydict.copy()
    vars = []
    effective_vars = []
    effective_idx_cut = []
    # index = 0
    count_dict = {}
    # regions = cutset_regions.copy()
    # print("dict",regions.exprdict)
    # 下面这里很奇怪, 但是其实就是把矩阵形式的不等式转化为region形式, 保存regions.
    for row in effective_constraints:
        terms = []
        for i in range(len(row) - 1):
            coef = row[i]
            if coef != 0:
                if entropydict.get_keys_by_value(i) != None:
                    term_x = entropydict.get_keys_by_value(i)[0]
                    var_str = term_x.split(",")
                    if term_x in count_dict:
                        count_dict[term_x] += 1
                    else:
                        count_dict[term_x] = 1
                    if var_str not in vars:
                        vars.append(var_str) # 添加有效不等式中的变量
                    term_x = Comp.jes(term_x)
                    term = Term(x=[term_x.copy()],coef=int(coef),termtype=TermType.H)
                    terms.append(term) 
        expr = Expr(terms, eqtype="ge", value=row[-1])
        expr.sort_terms()
        # print("expr",expr)
        regions.append_expr(expr)
    # for expr in regions.exprs:
    #     print(expr)
    effective_vars = vars[:]
    # print("effective vars",vars)
    print("len of effecitve vars",len(vars))
    count_dict = sorted(count_dict.items(), key=lambda item: item[1],reverse=True)
    count_dict = dict(count_dict)
    print(count_dict)

    # 生成新vars
    print("2.生成新vars")
    # 方式1：effective vars的子集
    # generate_vars = Iutils.generate_random_subsets(effective_vars,generate_size,2,episode+3)
    
    # 方式2：ori vars的交并集
    generate_vars = []
    for i in range(len(ori_vars)):
        for j in range(i + 1, len(ori_vars)):
            union = list(set(ori_vars[i]) | set(ori_vars[j]))
            if len(union) <= episode+1:
                # print("lenunion",len(union))
                union.sort()
                generate_vars.append(union)
            ins = list(set(ori_vars[i]) & set(ori_vars[j]))
            if ins:
                ins.sort()
                generate_vars.append(ins)

    # 方式3：effctive vars的交并集
    # generate_vars = []
    # for i in range(len(effective_vars)):
    #     for j in range(i + 1, len(effective_vars)):
    #         union = list(set(effective_vars[i]) | set(effective_vars[j]))
    #         if len(union) <= episode+1:
    #             # print("lenunion",len(union))
    #             union.sort()
    #             generate_vars.append(union)
    #         ins = list(set(effective_vars[i]) & set(effective_vars[j]))
    #         if ins:
    #             ins.sort()
    #             generate_vars.append(ins)
    
    if generate_size > len(generate_vars): # 补齐, 防止越界
        generate_size = len(generate_vars)
    random_indices = np.random.choice(len(generate_vars), size=generate_size, replace=False)
    selected_vars = [generate_vars[i] for i in random_indices]
    # selected_vars = generate_vars[:]

    for var in selected_vars:
        if var not in vars:
            vars.append(var)
    print(len(vars))
    # for var in single_vars:
    #     if [var] not in vars:
    #         vars.append([var])
    # print(len(vars))


    # 随机引入子集
    result_subsets = Iutils.generate_random_subsets(single_vars, subset_size, 2, episode+4)
    for subset in result_subsets:
        if subset not in vars and subset not in oriall_vars:
            vars.append(subset)

    # 显示vars构成
    print("ori vars, 其实没有用到这个",len(ori_vars))
    # print(ori_vars)
    print("generate vars, 有效变量的union和intersection",len(generate_vars))
    # print(generate_vars)
    print("selected vars, 从上面的变量中随机选择了30个",len(selected_vars))
    # print(selected_vars)
    print("random vars, 从单变量中随机组合的subsets",len(result_subsets))
    # print(result_subsets)
    print("number of varibles, 综合之后的总的变量:",len(vars))

    # 对vars进行封闭集和对称性处理，生成entropydict
    print("3.对vars进行封闭集和对称性处理，生成entropydict")
    expand_vars = vars[:]
    combs,combinations = Iutils.generate_combs(single_vars,comb_size)
    Iutils.preprocessing_combs(vars,single_vars,expand_vars,combs)
    for var in necessary_vars:
        if var not in expand_vars:
            expand_vars.append(var)
    print("before symmetrize num of expanded vars:",len(expand_vars))
    entropydict = EntropyEqDict()
    # Xrvs_cons = []
    # Wrvs_cons = []
    print("all_before",len(entropydict_all.redict))
    before_sym_time = time.time()
    Iutils.symmetrize_by_dict(user_perm,file_perm,expand_vars,entropydict,entropydict_all)
    print("-----------symmetrize time",time.time()-before_sym_time,"-----------")
    print("after symmetreize num of expanded vars:",len(entropydict.redict))
    print("all_after",len(entropydict_all.redict))
    # Iutils.symmetrize(user_perm,file_perm,expand_vars,entropydict,Xrvs_cons,Wrvs_cons)
    # print(entropydict)

    # 问题约束
    print("4.问题约束")
    Iutils.problem_constraints_process(Wkey,entropydict)
    entropydict.regenerate_keys()
    Iutils.problem_constraints_process(Wkey,entropydict_all)
    entropydict_all.regenerate_keys()
    print("number of problem variebles",len(entropydict.redict))
    print("number of all the variebles",len(entropydict.eqdict))

    # 生成不等式集，并合并相同不等式
    # print("5.生成不等式集，并合并相同不等式")
    # Iutils.generate_inequalities_combs(vars,entropydict,regions,combinations)
    # print("before reducing",len(regions.exprs))
    # regions.reduce_redundant_expr()
    # print("number of exprs",len(regions.exprs))

    print("5.生成不等式集，并合并相同不等式")
    regions_candidate = Region.empty()
    Iutils.generate_inequalities_combs(vars,entropydict,regions_candidate,combinations)
    #print("before reducing",len(regions.exprs))
    regions_candidate.reduce_redundant_expr()
    #print("number of exprs",len(regions.exprs))
    for i in range(len(regions_candidate.exprs)):
        regions.append_expr(regions_candidate.exprs[i])
    regions.reduce_redundant_expr()

    # 生成不等式矩阵
    print("6.生成不等式矩阵")
    ine_constraints = Regions2Matrix(entropydict, regions)
    ent_num = len(entropydict.redict) + 3
    ine_constraints,prob_cons_num = AddProblemConstraints2Matrix(Xrvs_cons, Wrvs_cons, entropydict, ine_constraints, ent_num)
    ent_num -= 1 # 实际变量数量
    

    # 问题求解
    plot_data = []
    effective_idx_gurobi = []
    all_eff_indx = set()
    result_slope = []
    ori_slope = []
    M_space = np.linspace(0,N,point_num*N+1)
    dual_value = []
    # print(M_space)
    t_solve = 0
    # print("shape of ine_constraints",ine_constraints.shape)
    for M_value in M_space:
        s = time.perf_counter()
        # 根据M_value更新约束矩阵，添加等式约束
        ine_constraints = list(ine_constraints[:-1])
        row = [0] * (ent_num + 1)
        row[-3] = 1
        row[-1] = M_value
        ine_constraints.append(row)

        ine_constraints = np.array(ine_constraints)
        if M_value == 0:
            print("shape of ine_constraints",ine_constraints.shape)
        ine_constraints = ine_constraints.astype(np.float64)

        ori_obj_coef = np.zeros(ent_num)
        ori_obj_coef[-1] = 1
        # print("ine_constraints")
        # print(ine_constraints)
        # print("ori_obj_coef",ori_obj_coef)

        # # 更新对偶问题约束矩阵
        expr_num = ine_constraints.shape[0] - 1
        trans_ine_cons = ine_constraints.T[:-1] # 对偶问题的约束矩阵 是原约束矩阵的转置
        dual_obj_coef = ine_constraints[:,-1] # 原约束的常量 是对偶问题目标函数的系数
        trans_ine_cons = np.hstack((trans_ine_cons, ori_obj_coef.T.reshape(-1, 1))) # 原目标函数的系数，是对偶问题约束的常量
        # print("shape",trans_ine_cons.shape)
        # print("trans_ine_cons")
        # print(trans_ine_cons)
        # print("dual_obj_coef",dual_obj_coef)

        # 求解原LP问题
        before_lp_time = time.time()
        result = gurobi_solver()
        if type(result) == list:
            bad = Region.empty()
            for ine in result:
                idx = int(ine[1:])
                print(f"type:{type(idx)},value:{idx}")
                terms = []
                row = ine_constraints[idx]
                for i in range(len(row) - 1):
                    coef = row[i]
                    if coef != 0:
                        if entropydict.get_keys_by_value(i) != None:
                            term_x = entropydict.get_keys_by_value(i)[0]
                            var_str = term_x.split(",")
                            if term_x in count_dict:
                                count_dict[term_x] += 1
                            else:
                                count_dict[term_x] = 1
                            if var_str not in vars:
                                vars.append(var_str) # 添加有效不等式中的变量
                            term_x = Comp.jes(term_x)
                            term = Term(x=[term_x.copy()],coef=int(coef),termtype=TermType.H)
                            terms.append(term) 
                expr = Expr(terms, eqtype="ge", value=row[-1])
                expr.sort_terms()
                # print("expr",expr)
                bad.append_expr(expr)
            for expr in bad.exprs:
                print(expr)

        if result > 0:
            # 求解对偶LP问题
            solution_values, effective_idx_dual = dual_solver()
            if solution_values is not None:
                dual_value.append(list(solution_values.values()))

            # print("effective_indices",effective_idx_dual)
            if effective_idx_dual is not None:
                all_eff_indx = all_eff_indx.union(set(effective_idx_dual))
            effective_idx_dual = sorted(list(all_eff_indx))
        plot_data.append((M_value, result))
        e = time.perf_counter()
        t = e - s
        t_solve += t

    print("-------------Lp time: ",time.time()-before_lp_time,"-------------")
    print("solve time",t_solve)
    if len(dual_value) > 0:
        effective_idx_cut = find_min_effective_indices(dual_value)
        print(f"dual:{len(effective_idx_dual)}")
        print(effective_idx_dual)
        print(f"gurobi:{len(effective_idx_gurobi)}")
        print(effective_idx_gurobi)
        print(f"cut:{len(effective_idx_cut)}")
        print(effective_idx_cut)
    # 绘制图像
 
    # plot the cut-set bound
    Iutils.plot_cutset_bound(N,K,point_num)
    Iutils.plot_inner_bound(N,K)


    x = [item[0] for item in plot_data]
    y = [item[1] for item in plot_data]
    result_slope = Iutils.compute_slopes(x,y)
    point_x = []
    point_y = []
    for i in range(1,len(result_slope)):
        if result_slope[i-1] != result_slope[i]:
            point_x.append(x[i])
            point_y.append(y[i])
    plt.scatter(point_x,point_y,color="red")
    
    for xi, yi in zip(point_x, point_y):
        label = f"({round(xi,3)}, {round(yi,3)})"
        plt.annotate(label,  
                    (xi, yi),  
                    textcoords="offset points",  
                    xytext=(30, 5),  
                    ha='center')  
    plt.plot(x, y, color='red', linewidth=2, label='computed outer bound')

    
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.xlabel("M")
    plt.ylabel("R")
    plt.legend()
    plt.title(f"Case(N,K)=({N},{K}),episode{episode}\nX_type={X_combinations}\nvars:{len(vars)},cons:{ine_constraints.shape}")
    plt.show()

    print(f"from start to now takes {time.time() - start_time}s")
    rewarder.update_reward_list(plot_data)
    print("discount_reward_list",rewarder.discount_reward_list)
    print(x)
    print(y)

episode 1
[]
[-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5]
oriall vars 0
same times 0
1.更新regions、vars
number of effective exprs: 4
len of effecitve vars 5
{'Z1': 3, 'X12': 2, 'X12,X21': 2, 'W1,W2': 2, 'X12,Z1': 2}
2.生成新vars
10
ori vars, 其实没有用到这个 9
generate vars, 有效变量的union和intersection 21
selected vars, 从上面的变量中随机选择了30个 21
random vars, 从单变量中随机组合的subsets 26
number of varibles, 综合之后的总的变量: 24
3.对vars进行封闭集和对称性处理，生成entropydict
before symmetrize num of expanded vars: 59
all_before 5
-----------symmetrize time 0.0009999275207519531 -----------
after symmetreize num of expanded vars: 21
all_after 21
4.问题约束
number of problem variebles 11
number of all the variebles 57
5.生成不等式集，并合并相同不等式
6.生成不等式矩阵
shape of ine_constraints (43, 14)
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22635.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11260H @ 2.60GHz, instruction set [SSE2|AVX|AVX

C:\Users\user\AppData\Local\Temp\ipykernel_18008\4208771157.py:319: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



CPU model: 11th Gen Intel(R) Core(TM) i5-11260H @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 46 rows, 13 columns and 122 nonzeros
Model fingerprint: 0xc2b96419
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 2e+00]
Presolve removed 15 rows and 2 columns
Presolve time: 0.01s
Presolved: 31 rows, 11 columns, 126 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.0000000e+29   3.000000e+30   5.000000e-01      0s
       8    7.5000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.500000000e-01
Optimal value: 0.75
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22635.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11260H @ 2.60GHz, instruction set [SS

In [24]:
entropydict.regenerate_keys()
print(entropydict.redict)

{0: ['W1,W2', 'W1,W2,X12', 'W1,W2,X21', 'W1,W2,Z1', 'W1,W2,Z2', 'W1,X12,Z2', 'W2,X12,Z1', 'W2,X21,Z2', 'X12,X21,Z1', 'X12,X21,Z2', 'X12,Z1,Z2', 'X21,Z1,Z2', 'W1,W2,X12,X21', 'W1,W2,X12,Z1', 'W1,W2,X12,Z2', 'W1,W2,X21,Z2', 'W1,W2,Z1,Z2', 'W1,X12,X21,Z2', 'W1,X12,Z1,Z2', 'W2,X12,X21,Z1', 'W2,X12,X21,Z2', 'W2,X12,Z1,Z2', 'W2,X21,Z1,Z2', 'X12,X21,Z1,Z2', 'W1,W2,X12,X21,Z1', 'W1,W2,X12,X21,Z2', 'W1,W2,X12,Z1,Z2', 'W1,W2,X21,Z1,Z2', 'W1,X12,X21,Z1,Z2', 'W2,X12,X21,Z1,Z2', 'W1,W2,X12,X21,Z1,Z2'], 1: ['X12'], 2: ['Z1', 'Z2'], 3: ['W1,X12,Z1', 'W1,X21,Z2', 'W2,X12,Z2', 'W2,X21,Z1'], 4: ['Z1,Z2'], 5: ['W1,Z1', 'W1,Z2', 'W2,Z1', 'W2,Z2'], 6: ['W1,X12,X21', 'W2,X12,X21'], 7: ['W1', 'W2'], 8: ['W1,Z1,Z2', 'W2,Z1,Z2'], 9: ['W1,X12', 'W1,X21', 'W2,X12', 'W2,X21'], 10: ['X12,Z1', 'X12,Z2', 'X21,Z1', 'X21,Z2'], 11: ['X12,X21']}
